In [4]:
!cp /data/shpx/notebooks/ocarmeli/Reordering_HP_CDL_mid_iid.ipynb /data/shpx/notebooks/olivyatan/Reordering_HP_CDL_mid_iid_ortal_copy.ipynb

cp: cannot stat '/data/shpx/notebooks/ocarmeli/Reordering_HP_CDL_mid_iid.ipynb': No such file or directory


In [2]:
!cp /data/shpx/notebooks/ocarmeli/Reordering_HP_CDL_mid_iid.ipynb

cp: missing destination file operand after '/data/shpx/notebooks/ocarmeli/Reordering_HP_CDL_mid_iid.ipynb'
Try 'cp --help' for more information.


In [3]:
pwd

'/data/shpx/notebooks/olivyatan'

In [6]:
ls /data/shpx/notebooks/ocarmeli/Reordering_HP_CDL_mid_iid.ipynb

ls: cannot access '/data/shpx/notebooks/ocarmeli/Reordering_HP_CDL_mid_iid.ipynb': No such file or directory


In [1]:
import os
import pickle
import socket
import sys
from datetime import date, timedelta
from glob import glob

import pandas as pd
import pyspark.sql.functions as func
import xgboost as xgb
from fsspec.implementations import hdfs
from pretrainer import *
from pretrainer import *
from pyspark.sql import Row, SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.functions import udf
from pyspark.sql.types import (
    ArrayType,
    DoubleType,
    LongType,
    StringType,
    StructField,
    StructType,
)
from pyspark.sql.window import Window
from sklearn.feature_selection import GenericUnivariateSelect, chi2, f_classif
from sklearn.linear_model import LogisticRegression

In [2]:
import os
from datetime import date
import socket
import sys
from pyspark.sql import SparkSession

def load_spark(queue, config={}):
    """
    params:
        queue - queue of spark jobs, e.g. 'hdlq-struct-default'
        config - dictionary to overwrite default params
    returns:
        spark session object
    """
    
    sys.path.append("/apache/spark3.1/python/lib/py4j-0.10.7-src.zip")
    sys.path.append('/apache/spark3.1/python')
    os.environ["SPARK_HOME"] = "/apache/spark3.1/"
    
    os.environ["PYSPARK_PYTHON"] = "/usr/share/anaconda3/python3.7/bin/python"
    sessionBuilder = (SparkSession.builder.appName(os.environ['KRYLOV_PRINCIPAL'])
        .master("yarn")
        .config("spark.driver.maxResultSize", "12g")
        .config("spark.driver.host", socket.gethostbyname(socket.gethostname()))
        .config("spark.driver.port", "30202")
        .config("spark.executor.memoryOverhead", "24g")
        .config("spark.executor.cores", "8")
        .config("spark.driver.memory", "36g")
        .config("spark.executor.memory", "36g")
        .config("spark.rdd.compress", True)
        .config("spark.network.timeout", "600s")
        .config("spark.executor.heartbeatInterval", "300s")
        .config("spark.sql.broadcastTimeout", "1200s")
        .config("spark.dynamicAllocation.minExecutors", 0)
        .config("spark.dynamicAllocation.initialExecutors", 10)
        .config("spark.dynamicAllocation.maxExecutors", 512)
        .config("spark.sql.shuffle.partitions", 512)
        .config("spark.kryoserializer.buffer.max", "1g")
        .config("spark.yarn.queue", queue)
        .config("spark.speculation", False)
        .config("spark.rpc.message.maxSize", 1024)
        .config("spark.pyspark.python", "/usr/share/anaconda3/python3.7/bin/python")
    )
    
    for k,v in config.items():
        sessionBuilder.config(k,v)
        
    spark = sessionBuilder.enableHiveSupport().getOrCreate()
    spark.sparkContext.setLogLevel("ERROR")
    spark.sparkContext.setCheckpointDir("checkpoint/")
    

    
    print("The executors' logs link:")
    print(f'https://apollo-rno-rm-2.vip.hadoop.ebay.com:50030/proxy/{spark.sparkContext.applicationId}')
    return spark

In [3]:
spark = load_spark(queue='hddq-exprce-perso-high-mem')

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/apache/releases/hadoop-2.7.3.2.6.4.2.0.38/share/hadoop/common/lib/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/apache/releases/hbase-1.1.2.2.6.4.1.0.3/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]


2023-07-06T12:38:06.802+0000: [GC (Metadata GC Threshold) 
Desired survivor size 89128960 bytes, new threshold 7 (max 15)
[PSYoungGen: 304494K->19333K(611840K)] 304494K->19373K(2010112K), 0.0212121 secs] [Times: user=0.06 sys=0.01, real=0.02 secs] 
2023-07-06T12:38:06.823+0000: [Full GC (Metadata GC Threshold) [PSYoungGen: 19333K->0K(611840K)] [ParOldGen: 40K->18602K(882688K)] 19373K->18602K(1494528K), [Metaspace: 20275K->20275K(22528K)], 0.0392423 secs] [Times: user=0.10 sys=0.00, real=0.04 secs] 


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2023-07-06T12:38:08.351+0000: [GC (Metadata GC Threshold) 
Desired survivor size 89128960 bytes, new threshold 7 (max 15)
[PSYoungGen: 315321K->15977K(611840K)] 333924K->34587K(1494528K), 0.0221982 secs] [Times: user=0.04 sys=0.01, real=0.03 secs] 
2023-07-06T12:38:08.373+0000: [Full GC (Metadata GC Threshold) [PSYoungGen: 15977K->0K(611840K)] [ParOldGen: 18610K->23759K(1461760K)] 34587K->23759K(2073600K), [Metaspace: 33666K->33666K(36864K)], 0.0468869 secs] [Times: user=0.08 sys=0.00, real=0.04 secs] 
2023-07-06T12:38:10.346+0000: [GC (Metadata GC Threshold) 
Desired survivor size 89128960 bytes, new threshold 7 (max 15)
[PSYoungGen: 476355K->23544K(611840K)] 500114K->47311K(2073600K), 0.0222566 secs] [Times: user=0.03 sys=0.01, real=0.02 secs] 
2023-07-06T12:38:10.368+0000: [Full GC (Metadata GC Threshold) [PSYoungGen: 23544K->0K(611840K)] [ParOldGen: 23767K->35987K(2098176K)] 47311K->35987K(2710016K), [Metaspace: 56409K->56395K(59392K)], 0.0903531 secs] [Times: user=0.18 sys=0.00, r

23/07/06 12:38:11 WARN HiveConf: DEPRECATED: hive.metastore.ds.retry.* no longer has any effect.  Use hive.hmshandler.retry.* instead
23/07/06 12:38:11 WARN HiveConf: HiveConf of name hive.metastore.local does not exist
23/07/06 12:38:11 WARN HiveConf: HiveConf of name hive.enforce.sorting does not exist
23/07/06 12:38:11 WARN HiveConf: HiveConf of name hive.server2.proxyuser.hue.groups does not exist
23/07/06 12:38:11 WARN HiveConf: HiveConf of name hive.server2.proxyuser.hue.hosts does not exist
23/07/06 12:38:11 WARN HiveConf: HiveConf of name hive.metastore.ds.retry.interval does not exist
23/07/06 12:38:11 WARN HiveConf: HiveConf of name hive.enforce.bucketing does not exist
23/07/06 12:38:11 WARN HiveConf: HiveConf of name hive.metastore.ds.retry.attempts does not exist
23/07/06 12:38:11 WARN HiveConf: HiveConf of name hive.server2.enable.impersonation does not exist


2023-07-06T12:38:12.835+0000: [GC (Allocation Failure) 
Desired survivor size 89128960 bytes, new threshold 7 (max 15)
[PSYoungGen: 524800K->24526K(726016K)] 560787K->60530K(2824192K), 0.0273375 secs] [Times: user=0.03 sys=0.01, real=0.02 secs] 


23/07/06 12:38:13 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
23/07/06 12:38:42 WARN YarnSchedulerBackend$YarnSchedulerEndpoint: Attempted to request executors before the AM has registered!


2023-07-06T12:38:43.495+0000: [GC (Allocation Failure) 
Desired survivor size 37224448 bytes, new threshold 8 (max 15)
[PSYoungGen: 694222K->35404K(756736K)] 730226K->71415K(2854912K), 0.0476531 secs] [Times: user=0.06 sys=0.05, real=0.04 secs] 
2023-07-06T12:38:43.877+0000: [GC (Metadata GC Threshold) 
Desired survivor size 38797312 bytes, new threshold 9 (max 15)
[PSYoungGen: 101497K->29545K(858624K)] 137508K->65565K(2956800K), 0.0216038 secs] [Times: user=0.04 sys=0.01, real=0.02 secs] 
2023-07-06T12:38:43.898+0000: [Full GC (Metadata GC Threshold) [PSYoungGen: 29545K->0K(858624K)] [ParOldGen: 36019K->50320K(2751488K)] 65565K->50320K(3610112K), [Metaspace: 94250K->94245K(100352K)], 0.1403472 secs] [Times: user=0.21 sys=0.03, real=0.14 secs] 
The executors' logs link:
https://apollo-rno-rm-2.vip.hadoop.ebay.com:50030/proxy/application_1687661240187_1208896


In [4]:
spark

In [5]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 10)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

### Load ab test data: 

In [6]:
import pandas as pd
import datetime
import os

from pyspark.sql import functions as F
from pyspark.sql import types as T

In [7]:
def get_data_path_dir(arr, allow_asterisk=True):
    if not arr:
        assert allow_asterisk, "missing partition for path"
        return "*"
    elif len(arr) > 1:
        return "{" + ",".join([str(elem) for elem in arr]) + "}"
    else:
        return str(arr[0])


def get_madcdl_path(from_date, to_date, placements):
    return os.path.join(
        "/apps/b_adv/plmerch/cdl",
        get_data_path_dir(
            [datetime.datetime.strftime(date, '%Y/%m/%d') for date in pd.date_range(from_date, to_date)],
            allow_asterisk=False
        ),
        "detail_gmv_part/*",
        "plmt=" + get_data_path_dir(placements)
    )

#madcdl us 
def get_madcdl(spark, from_date, to_date, placements=None):
    df_mad = spark.read.parquet(get_madcdl_path(from_date, to_date, placements)) \
        .filter(F.col("ICFBOT") == "00") \
        .filter(F.col("SITE_ID") == 0) \
        .filter(F.length(F.col("EUID")) == 0) \
        .withColumn("ts", F.to_timestamp(F.col("MERCH_DATE"), "yyyy/MM/dd HH:mm:ss")) \
        .withColumn("eventTimestamp", F.col("ts").cast(T.LongType()) * 1000) \

    return df_mad

In [8]:
madcdl = get_madcdl(
    spark,
    datetime.date(2023, 6, 6),
    datetime.date(2023, 6, 20),
    [101251, 101259, 101260, 101551, 101574, 101575, 101576]
)

In [9]:
madcdl.createOrReplaceTempView("madcdl")

2023-07-06T12:38:57.951+0000: [GC (Allocation Failure) 
Desired survivor size 47185920 bytes, new threshold 8 (max 15)
[PSYoungGen: 822272K->37877K(860160K)] 872592K->94815K(3611648K), 0.0905976 secs] [Times: user=0.08 sys=0.13, real=0.09 secs] 


In [10]:
sneakers_abtest_ios_android_web_items = spark.sql("""
 select * from ( SELECT 
      TRKS,
      user_id ,
	  SITE_ID,
      plmt,
	  dt,
	  meid,
	  Experience,
	  SURFACE_PLMT_IMP,
	  ALGO_OUTPUT_ITEM_CNT,
	  split(outitm, ':') [0]  AS itemId, 
	  split(outitm, ':') [21]  AS rank, 
	  split(outitm, ':') [15]  AS leafCategoryId, 
	  split(outitm, ':') [2]  AS price,  
	  split(outitm, ':') [9]  AS clicks,
	  clicks as plmt_clicks,
	  purchase_1d as plmt_purchase_1d, 
	  gmv_sum_1d as plmt_gmv_sum_1d,
	  ALGO_OUTPUT_ITEMS,
	 case when  Experience in('Browser: mWeb','Browser: Core site') then 'Browser'
	 when  Experience in('Apps: iPhone','Apps: Android') then 'Apps' end as platform ,

	CASE
    WHEN TRKS IN (250480) AND dt BETWEEN '20230606' AND '20230613' AND Experience IN ('Browser: mWeb','Browser: Core site') THEN 'treatment_web'
    WHEN TRKS IN (250481) AND dt BETWEEN '20230606' AND '20230613' AND Experience IN ('Browser: mWeb','Browser: Core site') THEN 'control_web'
    WHEN TRKS IN (250484) AND dt BETWEEN '20230606' AND '20230620' AND Experience = 'Apps: Android' THEN 'treatment_Android'
    WHEN TRKS IN (250485) AND dt BETWEEN '20230606' AND '20230620' AND Experience = 'Apps: Android' THEN 'control_Android'
    WHEN TRKS IN (250482) AND dt BETWEEN '20230606' AND '20230620' AND Experience = 'Apps: iPhone' THEN 'treatment_iOs'
    WHEN TRKS IN (250483) AND dt BETWEEN '20230606' AND '20230620' AND Experience = 'Apps: iPhone' THEN 'control_iOs'
 	END AS test_group,
	
	   CASE
    WHEN (TRKS IN (250480) AND dt BETWEEN '20230606' AND '20230613' AND Experience IN ('Browser: mWeb','Browser: Core site'))
         OR (TRKS IN (250484) AND dt BETWEEN '20230606' AND '20230620' AND Experience = 'Apps: Android')
         OR (TRKS IN (250482) AND dt BETWEEN '20230606' AND '20230620' AND Experience = 'Apps: iPhone') THEN 'treatment'
    WHEN (TRKS IN (250481) AND dt BETWEEN '20230606' AND '20230613' AND Experience IN ('Browser: mWeb','Browser: Core site'))
         OR (TRKS IN (250485) AND dt BETWEEN '20230606' AND '20230620' AND Experience = 'Apps: Android')
         OR (TRKS IN (250483) AND dt BETWEEN '20230606' AND '20230620' AND Experience = 'Apps: iPhone') THEN 'control'
  END AS treatment_control_agg

    FROM
    bpe_v.plmerch_data_v  LATERAL VIEW explode(`ALGO_OUTPUT_ITEMS`) outitm AS outitm 
    WHERE
	 SURFACE_PLMT_IMP=1 
	 and  plmt in (101251, 101259, 101260, 101551, 101574, 101575, 101576)
	 and TRKS in (250484, 250485,250482,250483 ,250480, 250481 ) --control, treatment, sneakers ios android web 
	 AND dt BETWEEN '20230606' AND '20230620'
     AND ICFBOT = '00' 
     AND length(EUID) = 0 
	 and SITE_ID in (0 ) -- us only 
	 and Experience in('Apps: iPhone','Apps: Android', 'Browser: mWeb','Browser: Core site')-- web and mobile 
	) as s  
	where leafCategoryId in ( 15709, 95672)
    
""")
sneakers_abtest_ios_android_web_items.cache()
sneakers_abtest_ios_android_web_items.show()

+------+----------+-------+------+--------+--------------------+------------------+----------------+--------------------+------------+----+--------------+-----+------+-----------+----------------+---------------+--------------------+--------+-------------+---------------------+
|  TRKS|   user_id|SITE_ID|  plmt|      dt|                meid|        Experience|SURFACE_PLMT_IMP|ALGO_OUTPUT_ITEM_CNT|      itemId|rank|leafCategoryId|price|clicks|plmt_clicks|plmt_purchase_1d|plmt_gmv_sum_1d|   ALGO_OUTPUT_ITEMS|platform|   test_group|treatment_control_agg|
+------+----------+-------+------+--------+--------------------+------------------+----------------+--------------------+------------+----+--------------+-----+------+-----------+----------------+---------------+--------------------+--------+-------------+---------------------+
|250481|2410828769|      0|101251|20230606|c6f406de525946878...|Browser: Core site|               1|                  10|195778208439|   1|         15709| 8000|   

In [11]:
sneakers_transactions = spark.sql("""

  SELECT 
      TRKS,
      user_id,
	  SITE_ID,
      plmt,
	  dt,
	  meid,
	  SURFACE_PLMT_IMP,
	--  split(GMV_1D1, ':') [0]  AS userId,
	  split(GMV_1D1, ':') [1]  AS itemId, 
	--  split(GMV_1D1, ':') [2]  AS transactionId,  
	  PURCHASE_1D
    FROM
    bpe_v.plmerch_data_v  LATERAL VIEW explode(`GMV_1D`)  AS GMV_1D1
	 WHERE
	 SURFACE_PLMT_IMP=1 
	  and purchase_1d >0 
	 and  plmt in (101251, 101259, 101260, 101551, 101574, 101575, 101576)
	 and TRKS in (250484, 250485,250482,250483 ,250480, 250481 ) --control, treatment, sneakers ios android web 
	 AND dt BETWEEN '20230606' AND '20230620'
     AND ICFBOT = '00' 
     AND length(EUID) = 0 
	 and SITE_ID in (0 ) -- us only 
	 and Experience in('Apps: iPhone','Apps: Android', 'Browser: mWeb','Browser: Core site')-- web and mobile 

	group by 1,2,3,4,5,6,7,8,9
    
""")
sneakers_transactions.cache()
sneakers_transactions.show()

2023-07-06T12:39:30.606+0000: [GC (Allocation Failure) 
Desired survivor size 64487424 bytes, new threshold 7 (max 15)
[PSYoungGen: 860149K->46058K(1178624K)] 917087K->128255K(3930112K), 0.0754485 secs] [Times: user=0.14 sys=0.08, real=0.07 secs] 


2023-07-06T12:39:44.489+0000: [GC (Allocation Failure) 
Desired survivor size 82837504 bytes, new threshold 6 (max 15)
[PSYoungGen: 1178602K->62951K(1209856K)] 1260799K->168152K(3961344K), 0.1038639 secs] [Times: user=0.17 sys=0.13, real=0.11 secs] 


2023-07-06T12:39:53.740+0000: [GC (Allocation Failure) 
Desired survivor size 92274688 bytes, new threshold 7 (max 15)
[PSYoungGen: 1209831K->74932K(1441280K)] 1315032K->180140K(4192768K), 0.0679280 secs] [Times: user=0.12 sys=0.08, real=0.07 secs] 


2023-07-06T12:39:58.914+0000: [GC (Allocation Failure) 
Desired survivor size 105381888 bytes, new threshold 8 (max 15)
[PSYoungGen: 1435316K->89739K(1450496K)] 1540524K->194948K(4201984K), 0.0641790 secs] [Times: user=0.13 sys=0.06, real=0.06 secs] 


2023-07-06T12:40:02.645+0000: [GC (Allocation Failure) 
Desired survivor size 121634816 bytes, new threshold 7 (max 15)
[PSYoungGen: 1450123K->102890K(1717760K)] 1555332K->209610K(4469248K), 0.0758401 secs] [Times: user=0.17 sys=0.05, real=0.07 secs] 


...

2023-07-06T12:40:09.656+0000: [GC (Allocation Failure) 
Desired survivor size 147324928 bytes, new threshold 7 (max 15)
[PSYoungGen: 1733621K->98289K(2149888K)] 1842350K->244504K(4901376K), 0.0932484 secs] [Times: user=0.20 sys=0.08, real=0.09 secs] 


2023-07-06T12:40:13.394+0000: [GC (Allocation Failure) 
Desired survivor size 160432128 bytes, new threshold 8 (max 15)
[PSYoungGen: 2110961K->118740K(2156544K)] 2257176K->264962K(4908032K), 0.0857128 secs] [Times: user=0.24 sys=0.02, real=0.08 secs] 


2023-07-06T12:40:16.218+0000: [GC (Allocation Failure) 
Desired survivor size 178782208 bytes, new threshold 8 (max 15)
[PSYoungGen: 2131412K->141552K(2714624K)] 2277634K->287774K(5466112K), 0.0953047 secs] [Times: user=0.22 sys=0.07, real=0.10 secs] 


2023-07-06T12:40:19.221+0000: [GC (Allocation Failure) 
Desired survivor size 193462272 bytes, new threshold 7 (max 15)
[PSYoungGen: 2699504K->141855K(2732544K)] 2845726K->312090K(5484032K), 0.1088662 secs] [Times: user=0.24 sys=0.08, real=0.11 secs] 


+------+----------+-------+------+--------+--------------------+----------------+------------+-----------+
|  TRKS|   user_id|SITE_ID|  plmt|      dt|                meid|SURFACE_PLMT_IMP|      itemId|PURCHASE_1D|
+------+----------+-------+------+--------+--------------------+----------------+------------+-----------+
|250481|2532669094|      0|101251|20230610|d08f7edd1fad4a099...|               1|125937346525|          1|
|250484|1504524424|      0|101251|20230613|c97d7368fb3f446aa...|               1|374716107585|          1|
|250484|2360108253|      0|101251|20230613|c75dc3a445834d66a...|               1|166158220851|          1|
|250483|1620339212|      0|101251|20230620|14ff4d81abcc4ffaa...|               1|285328803466|          1|
|250485|  62217040|      0|101251|20230618|f5d5149fe92249dea...|               1|404323666367|          2|
|250481|   7594278|      0|101259|20230606|280209021c2644e59...|               1|166089674082|          1|
|250482|2442072185|      0|101251|202

In [12]:
sneakers_transactions.show()

+------+----------+-------+------+--------+--------------------+----------------+------------+-----------+
|  TRKS|   user_id|SITE_ID|  plmt|      dt|                meid|SURFACE_PLMT_IMP|      itemId|PURCHASE_1D|
+------+----------+-------+------+--------+--------------------+----------------+------------+-----------+
|250481|2532669094|      0|101251|20230610|d08f7edd1fad4a099...|               1|125937346525|          1|
|250484|1504524424|      0|101251|20230613|c97d7368fb3f446aa...|               1|374716107585|          1|
|250484|2360108253|      0|101251|20230613|c75dc3a445834d66a...|               1|166158220851|          1|
|250483|1620339212|      0|101251|20230620|14ff4d81abcc4ffaa...|               1|285328803466|          1|
|250485|  62217040|      0|101251|20230618|f5d5149fe92249dea...|               1|404323666367|          2|
|250481|   7594278|      0|101259|20230606|280209021c2644e59...|               1|166089674082|          1|
|250482|2442072185|      0|101251|202

In [25]:
sneakers_abtest_ios_android_web_items.createOrReplaceTempView("sneakers_abtest_ios_android_web_items")
sneakers_transactions.createOrReplaceTempView("sneakers_transactions")
sneakers_abtest_transactions = spark.sql("""

select 
	  i.TRKS,
	  i.test_group, 
	  i.treatment_control_agg,
      i.user_id ,
	  i.SITE_ID,
      i.plmt,
	  i.dt,
	  i.meid,
	  i.Experience,
	  i.platform, 
	  i.SURFACE_PLMT_IMP,
	  i.itemId, 
	  i.rank as production_rank, 
	  i.leafCategoryId, 
	  i.clicks,
	  i.price,
	  i.plmt_gmv_sum_1d,
	  i.plmt_purchase_1d  ,
	  i.plmt_clicks,
	  p.itemId as purchased_itemId,
      case when purchased_itemId is not null then 1 else 0 end as is_purchase


	from  sneakers_abtest_ios_android_web_items as i 
	left join sneakers_transactions as p on i.user_id=p.user_id and i.meid=p.meid and i.itemId=p.itemId
    
""")
sneakers_abtest_transactions.cache()
sneakers_abtest_transactions.show()

+------+-------------+---------------------+----------+-------+------+--------+--------------------+------------------+--------+----------------+------------+---------------+--------------+------+-----+---------------+----------------+-----------+----------------+-----------+
|  TRKS|   test_group|treatment_control_agg|   user_id|SITE_ID|  plmt|      dt|                meid|        Experience|platform|SURFACE_PLMT_IMP|      itemId|production_rank|leafCategoryId|clicks|price|plmt_gmv_sum_1d|plmt_purchase_1d|plmt_clicks|purchased_itemId|is_purchase|
+------+-------------+---------------------+----------+-------+------+--------+--------------------+------------------+--------+----------------+------------+---------------+--------------+------+-----+---------------+----------------+-----------+----------------+-----------+
|250481|  control_web|              control|2410828769|      0|101251|20230606|c6f406de525946878...|Browser: Core site| Browser|               1|195778208439|           

In [26]:
sneakers_abtest_transactions.count()

2023-07-06T14:13:38.208+0000: [GC (Allocation Failure) 
Desired survivor size 436731904 bytes, new threshold 15 (max 15)
[PSYoungGen: 5612032K->167045K(6031872K)] 5966857K->521878K(30137344K), 0.1108472 secs] [Times: user=0.29 sys=0.03, real=0.11 secs] 


2023-07-06T14:14:03.348+0000: [GC (Allocation Failure) 
Desired survivor size 467140608 bytes, new threshold 15 (max 15)
[PSYoungGen: 5779077K->237823K(6060544K)] 6133910K->592656K(30166016K), 0.1607529 secs] [Times: user=0.35 sys=0.13, real=0.16 secs] 


2023-07-06T14:14:16.131+0000: [GC (Allocation Failure) 
Desired survivor size 517472256 bytes, new threshold 15 (max 15)
[PSYoungGen: 5871871K->312432K(6090240K)] 6226704K->667265K(30195712K), 0.1715042 secs] [Times: user=0.45 sys=0.06, real=0.17 secs] 


71630905

### save test data

sneakers_abtest_data=spark.read.parquet('/user/b_perso/olivyatan/piyi/sneakers_abtest_transactions')

In [ ]:
sneakers_abtest_transactions.shape

### sample users: 

In [27]:
from pyspark.sql.functions import col

# Get the distinct user_ids
distinct_user_ids = sneakers_abtest_transactions.select("user_id").distinct()

# Calculate the fraction to sample (80%)
sample_fraction = 0.6

# Sample the user_ids
sampled_user_ids = distinct_user_ids.sample(fraction=sample_fraction, seed=42)



In [28]:
# Filter the original DataFrame to retain data for sampled users
sampled_sneakers_data = sneakers_abtest_transactions.join(sampled_user_ids, on="user_id", how="inner")

# Check the sampled DataFrame
sampled_sneakers_data.show()

2023-07-06T14:15:08.119+0000: [GC (Allocation Failure) 
Desired survivor size 529530880 bytes, new threshold 15 (max 15)
[PSYoungGen: 5943375K->241722K(6176768K)] 6298208K->596563K(30282240K), 0.1385312 secs] [Times: user=0.32 sys=0.08, real=0.14 secs] 


2023-07-06T14:15:10.039+0000: [GC (Allocation Failure) 
Desired survivor size 555745280 bytes, new threshold 15 (max 15)
[PSYoungGen: 5913146K->292924K(6188544K)] 6267987K->647774K(30294016K), 0.1345894 secs] [Times: user=0.40 sys=0.00, real=0.14 secs] 


2023-07-06T14:15:11.935+0000: [GC (Allocation Failure) 
Desired survivor size 637534208 bytes, new threshold 15 (max 15)
[PSYoungGen: 5964348K->458864K(6237696K)] 6319198K->813722K(30343168K), 0.3016008 secs] [Times: user=0.74 sys=0.16, real=0.30 secs] 


2023-07-06T14:15:14.015+0000: [GC (Allocation Failure) 
Desired survivor size 767033344 bytes, new threshold 14 (max 15)
[PSYoungGen: 6153840K->622580K(6317568K)] 6508698K->986125K(30423040K), 0.3699748 secs] [Times: user=1.00 sys=0.10, real=0.37 secs] 


2023-07-06T14:15:16.156+0000: [GC (Allocation Failure) 
Desired survivor size 927465472 bytes, new threshold 13 (max 15)
[PSYoungGen: 6317556K->749035K(6580224K)] 6681101K->1144395K(30685696K), 0.4976819 secs] [Times: user=1.13 sys=0.36, real=0.50 secs] 


2023-07-06T14:15:18.377+0000: [GC (Allocation Failure) 
Desired survivor size 1111490560 bytes, new threshold 12 (max 15)
[PSYoungGen: 6580203K->905725K(6736896K)] 6975563K->1321701K(30842368K), 0.4688086 secs] [Times: user=1.40 sys=0.00, real=0.47 secs] 


23/07/06 14:15:19 ERROR AsyncEventQueue: Dropping event from queue executorManagement. This likely means one of the listeners is too slow and cannot keep up with the rate at which tasks are being started by the scheduler.


+-------+------+-------------+---------------------+-------+------+--------+--------------------+------------------+--------+----------------+------------+---------------+--------------+------+-----+---------------+----------------+-----------+----------------+-----------+
|user_id|  TRKS|   test_group|treatment_control_agg|SITE_ID|  plmt|      dt|                meid|        Experience|platform|SURFACE_PLMT_IMP|      itemId|production_rank|leafCategoryId|clicks|price|plmt_gmv_sum_1d|plmt_purchase_1d|plmt_clicks|purchased_itemId|is_purchase|
+-------+------+-------------+---------------------+-------+------+--------+--------------------+------------------+--------+----------------+------------+---------------+--------------+------+-----+---------------+----------------+-----------+----------------+-----------+
| 938014|250480|treatment_web|            treatment|      0|101259|20230607|a3f3f56c05a74b4ca...|Browser: Core site| Browser|               1|334727550472|              1|       

In [29]:
sneakers_abtest_df = sampled_sneakers_data.toPandas()


2023-07-06T14:15:26.185+0000: [GC (Allocation Failure) 
Desired survivor size 1296564224 bytes, new threshold 13 (max 15)
[PSYoungGen: 6736893K->1026194K(7165952K)] 7152869K->1442178K(31271424K), 0.6667837 secs] [Times: user=1.52 sys=0.46, real=0.67 secs] 


2023-07-06T14:15:28.778+0000: [GC (Allocation Failure) 
Desired survivor size 1464336384 bytes, new threshold 14 (max 15)
[PSYoungGen: 7106706K->1078951K(7346688K)] 7522690K->1494935K(31452160K), 0.4772170 secs] [Times: user=1.43 sys=0.00, real=0.48 secs] 


2023-07-06T14:15:31.165+0000: [GC (Allocation Failure) 
Desired survivor size 1620049920 bytes, new threshold 15 (max 15)
[PSYoungGen: 7159463K->1141658K(7813120K)] 7575447K->1557642K(31918592K), 0.6242443 secs] [Times: user=1.54 sys=0.34, real=0.62 secs] 


2023-07-06T14:15:33.791+0000: [GC (Allocation Failure) 
Desired survivor size 1768423424 bytes, new threshold 15 (max 15)
[PSYoungGen: 7524762K->1211422K(7965184K)] 7940746K->1627406K(32070656K), 0.5375369 secs] [Times: user=1.60 sys=0.00, real=0.54 secs] 


2023-07-06T14:15:36.247+0000: [GC (Allocation Failure) 
Desired survivor size 1907884032 bytes, new threshold 15 (max 15)
[PSYoungGen: 7594526K->1277058K(8518144K)] 8010510K->1693042K(32623616K), 0.7275949 secs] [Times: user=1.74 sys=0.44, real=0.73 secs] 


2023-07-06T14:15:39.205+0000: [GC (Allocation Failure) 
Desired survivor size 1787822080 bytes, new threshold 15 (max 15)
[PSYoungGen: 8068226K->678060K(8654336K)] 8484210K->1094052K(32759808K), 0.2882802 secs] [Times: user=0.85 sys=0.01, real=0.29 secs] 


2023-07-06T14:15:58.119+0000: [GC (Allocation Failure) 
Desired survivor size 2084569088 bytes, new threshold 15 (max 15)
[PSYoungGen: 7469228K->1745919K(8828416K)] 7885220K->2456639K(32933888K), 0.5538947 secs] [Times: user=1.00 sys=0.64, real=0.56 secs] 
2023-07-06T14:16:03.057+0000: [GC (Allocation Failure) 
Desired survivor size 4294967296 bytes, new threshold 14 (max 15)
[PSYoungGen: 8622098K->2035705K(9118208K)] 9332818K->9156198K(33223680K), 3.2324170 secs] [Times: user=7.71 sys=1.97, real=3.23 secs] 
2023-07-06T14:16:10.738+0000: [GC (Allocation Failure) 
Desired survivor size 4294967296 bytes, new threshold 13 (max 15)
[PSYoungGen: 9016923K->3464673K(7659008K)] 16137416K->16319949K(31764480K), 3.6313658 secs] [Times: user=8.52 sys=2.37, real=3.63 secs] 
2023-07-06T14:16:16.951+0000: [GC (Allocation Failure) 
Desired survivor size 4294967296 bytes, new threshold 12 (max 15)
[PSYoungGen: 7658977K->4194298K(8388608K)] 20514253K->20015407K(32494080K), 2.3628914 secs] [Times: user=

In [30]:
sneakers_abtest_df.to_pickle('/data/shpx/notebooks/olivyatan/Unbiasing/sneakers_size_abtest_transactions_290623_sample.pkl' )

In [31]:
sneakers_abtest_df.shape

(42916591, 21)

In [32]:
sneakers_abtest_df.user_id.nunique()

463833

In [33]:
sneakers_abtest_df.columns

Index(['user_id', 'TRKS', 'test_group', 'treatment_control_agg', 'SITE_ID',
       'plmt', 'dt', 'meid', 'Experience', 'platform', 'SURFACE_PLMT_IMP',
       'itemId', 'production_rank', 'leafCategoryId', 'clicks', 'price',
       'plmt_gmv_sum_1d', 'plmt_purchase_1d', 'plmt_clicks',
       'purchased_itemId', 'is_purchase'],
      dtype='object')

In [34]:
!pwd

/data/shpx/notebooks/olivyatan/Unbiasing


In [35]:
sneakers_abtest_df.head(5)

,user_id,TRKS,test_group,treatment_control_agg,SITE_ID,plmt,dt,meid,Experience,platform,...,itemId,production_rank,leafCategoryId,clicks,price,plmt_gmv_sum_1d,plmt_purchase_1d,plmt_clicks,purchased_itemId,is_purchase
0,518278,250480,treatment_web,treatment,0,101251,20230612,0cf1375a0dc540eb8dfe785eb7dddeb8,Browser: Core site,Browser,...,175748430899,1,15709,,5995,0,0,[],None,0
1,518278,250480,treatment_web,treatment,0,101251,20230612,0cf1375a0dc540eb8dfe785eb7dddeb8,Browser: Core site,Browser,...,166156770983,2,15709,,3285,0,0,[],None,0
2,518278,250480,treatment_web,treatment,0,101251,20230612,0cf1375a0dc540eb8dfe785eb7dddeb8,Browser: Core site,Browser,...,166098940226,3,15709,,10999,0,0,[],None,0
3,518278,250480,treatment_web,treatment,0,101251,20230612,0cf1375a0dc540eb8dfe785eb7dddeb8,Browser: Core site,Browser,...,185744947926,4,15709,,9999,0,0,[],None,0
4,518278,250480,treatment_web,treatment,0,101251,20230612,0cf1375a0dc540eb8dfe785eb7dddeb8,Browser: Core site,Browser,...,175767689521,5,15709,,5899,0,0,[],None,0


2023-07-06T14:27:23.757+0000: [GC (Allocation Failure) 
Desired survivor size 525860864 bytes, new threshold 15 (max 15)
[PSYoungGen: 5082929K->2712K(5628928K)] 23931936K->18851743K(29734400K), 0.1016218 secs] [Times: user=0.20 sys=0.10, real=0.10 secs] 
2023-07-06T14:38:42.982+0000: [GC (System.gc()) 
Desired survivor size 485490688 bytes, new threshold 15 (max 15)
[PSYoungGen: 1102748K->5337K(5559296K)] 19951778K->18854391K(29664768K), 0.0629463 secs] [Times: user=0.14 sys=0.03, real=0.07 secs] 
2023-07-06T14:38:43.045+0000: [Full GC (System.gc()) [PSYoungGen: 5337K->0K(5559296K)] [ParOldGen: 18849054K->712339K(24105472K)] 18854391K->712339K(29664768K), [Metaspace: 154045K->154045K(165888K)], 1.1205272 secs] [Times: user=1.81 sys=0.57, real=1.12 secs] 
2023-07-06T15:08:42.982+0000: [GC (System.gc()) 
Desired survivor size 448266240 bytes, new threshold 15 (max 15)
[PSYoungGen: 3197799K->4568K(5129216K)] 3910139K->716915K(29234688K), 0.0248188 secs] [Times: user=0.04 sys=0.02, real=0.

23/07/06 15:25:08 ERROR YarnScheduler: Lost executor 1062 on hdc42-mcc10-01-0110-5409-046-tess0173.stratus.rno.ebay.com: Executor heartbeat timed out after 641043 ms
23/07/06 15:35:10 ERROR YarnScheduler: Lost executor 809 on hdc42-mcc10-01-0110-7605-012-tess0173.stratus.rno.ebay.com: Container from a bad node: container_e3815_1687661240187_1208896_01_001836 on host: hdc42-mcc10-01-0110-7605-012-tess0173.stratus.rno.ebay.com. Exit status: 154. Diagnostics: Container exited with a non-zero exit code 154
.


2023-07-06T15:38:42.982+0000: [GC (System.gc()) 
Desired survivor size 413138944 bytes, new threshold 15 (max 15)
[PSYoungGen: 2873798K->7007K(5493760K)] 3440253K->573471K(29599232K), 0.0249161 secs] [Times: user=0.05 sys=0.01, real=0.03 secs] 
2023-07-06T15:38:43.007+0000: [Full GC (System.gc()) [PSYoungGen: 7007K->0K(5493760K)] [ParOldGen: 566463K->566371K(24105472K)] 573471K->566371K(29599232K), [Metaspace: 154279K->154279K(165888K)], 0.6433531 secs] [Times: user=1.65 sys=0.00, real=0.64 secs] 


In [25]:
sneakers_abtest_df.columns

Index(['user_id', 'TRKS', 'SITE_ID', 'plmt', 'dt', 'meid', 'Experience',
       'SURFACE_PLMT_IMP', 'itemId', 'production_rank', 'leafCategoryId',
       'clicks', 'price', 'plmt_gmv_sum_1d', 'plmt_purchase_1d', 'plmt_clicks',
       'purchased_itemId', 'is_purchase'],
      dtype='object')

In [28]:
sneakers_abtest_df[sneakers_abtest_df.plmt_purchase_1d>0].head(4)

,user_id,TRKS,SITE_ID,plmt,dt,meid,Experience,SURFACE_PLMT_IMP,itemId,production_rank,leafCategoryId,clicks,price,plmt_gmv_sum_1d,plmt_purchase_1d,plmt_clicks,purchased_itemId,is_purchase
983921,2478517360,250482,0,101551,20230616,9f4be62eb40c4d46abe45c9e11101505,Apps: iPhone,1,175753744878,1,15709,,12963,13000,1,"[false::1686926799624::2478517360:295757165047::::::1686926361024::::::::::::::::::::2356359:162::::, false::1686926422993::2478517360:175753744878::::::1686926361024::::::::::::::::::::2356359:92::::, false::1686926779694::2478517360:175753744878::::::1686926361024::::::::::::::::::::2356359:135::::]",None,0
983922,2478517360,250482,0,101551,20230616,9f4be62eb40c4d46abe45c9e11101505,Apps: iPhone,1,385574021643,2,15709,,15000,13000,1,"[false::1686926799624::2478517360:295757165047::::::1686926361024::::::::::::::::::::2356359:162::::, false::1686926422993::2478517360:175753744878::::::1686926361024::::::::::::::::::::2356359:92::::, false::1686926779694::2478517360:175753744878::::::1686926361024::::::::::::::::::::2356359:135::::]",None,0
983923,2478517360,250482,0,101551,20230616,9f4be62eb40c4d46abe45c9e11101505,Apps: iPhone,1,194141507236,3,15709,,19999,13000,1,"[false::1686926799624::2478517360:295757165047::::::1686926361024::::::::::::::::::::2356359:162::::, false::1686926422993::2478517360:175753744878::::::1686926361024::::::::::::::::::::2356359:92::::, false::1686926779694::2478517360:175753744878::::::1686926361024::::::::::::::::::::2356359:135::::]",None,0
983924,2478517360,250482,0,101551,20230616,9f4be62eb40c4d46abe45c9e11101505,Apps: iPhone,1,134605253981,4,15709,,13000,13000,1,"[false::1686926799624::2478517360:295757165047::::::1686926361024::::::::::::::::::::2356359:162::::, false::1686926422993::2478517360:175753744878::::::1686926361024::::::::::::::::::::2356359:92::::, false::1686926779694::2478517360:175753744878::::::1686926361024::::::::::::::::::::2356359:135::::]",None,0


In [30]:
sneakers_abtest_df[sneakers_abtest_df.is_purchase>0].head(4)

,user_id,TRKS,SITE_ID,plmt,dt,meid,Experience,SURFACE_PLMT_IMP,itemId,production_rank,leafCategoryId,clicks,price,plmt_gmv_sum_1d,plmt_purchase_1d,plmt_clicks,purchased_itemId,is_purchase
983926,2478517360,250482,0,101551,20230616,9f4be62eb40c4d46abe45c9e11101505,Apps: iPhone,1,295757165047,6,15709,,14000,13000,1,"[false::1686926799624::2478517360:295757165047::::::1686926361024::::::::::::::::::::2356359:162::::, false::1686926422993::2478517360:175753744878::::::1686926361024::::::::::::::::::::2356359:92::::, false::1686926779694::2478517360:175753744878::::::1686926361024::::::::::::::::::::2356359:135::::]",295757165047,1
1405982,2357024171,250481,0,101251,20230611,38b93dc0e7e14995a72ab674907a66ad,Apps: Android,1,204360293323,1,95672,,3200,3200,1,[false:3664246316577:1686504246224::2357024171:204360293323::::::1686500665121::::::::::::::::::::2208336:640::::],204360293323,1
1775814,1180141384,250480,0,101259,20230611,f57ddf667bb24701b087e071c232e1cd,Browser: Core site,1,354825524272,1,15709,,11200,9500,1,[false:3664371943554:1686512131163::1180141384:354825524272::::::1686505112667::::::::::::::::::::2208336:3959::::],354825524272,1
1936960,1049297664,250482,0,101259,20230606,96bc9c18412e43c98de4bc00ea93cd39,Apps: iPhone,1,304962576825,3,95672,,10500,7000,1,"[false::1686099472905::1049297664:204274412500::::::1686099290528::::::::::::::::::::2356359:238::::, false::1686099497006::1049297664:304962576825::::::1686099290528::::::::::::::::::::2356359:291::::, false::1686099689357::1049297664:166014361490::::::1686099290528::::::::::::::::::::2356359:443::::, false::1686099623579::1049297664:304962576825::::::1686099290528::::::::::::::::::::2356359:407::::]",304962576825,1


In [32]:
sneakers_abtest_df[sneakers_abtest_df.meid=='9f4be62eb40c4d46abe45c9e11101505']

,user_id,TRKS,SITE_ID,plmt,dt,meid,Experience,SURFACE_PLMT_IMP,itemId,production_rank,leafCategoryId,clicks,price,plmt_gmv_sum_1d,plmt_purchase_1d,plmt_clicks,purchased_itemId,is_purchase
983921,2478517360,250482,0,101551,20230616,9f4be62eb40c4d46abe45c9e11101505,Apps: iPhone,1,175753744878,1,15709,,12963,13000,1,"[false::1686926799624::2478517360:295757165047::::::1686926361024::::::::::::::::::::2356359:162::::, false::1686926422993::2478517360:175753744878::::::1686926361024::::::::::::::::::::2356359:92::::, false::1686926779694::2478517360:175753744878::::::1686926361024::::::::::::::::::::2356359:135::::]",None,0
983922,2478517360,250482,0,101551,20230616,9f4be62eb40c4d46abe45c9e11101505,Apps: iPhone,1,385574021643,2,15709,,15000,13000,1,"[false::1686926799624::2478517360:295757165047::::::1686926361024::::::::::::::::::::2356359:162::::, false::1686926422993::2478517360:175753744878::::::1686926361024::::::::::::::::::::2356359:92::::, false::1686926779694::2478517360:175753744878::::::1686926361024::::::::::::::::::::2356359:135::::]",None,0
983923,2478517360,250482,0,101551,20230616,9f4be62eb40c4d46abe45c9e11101505,Apps: iPhone,1,194141507236,3,15709,,19999,13000,1,"[false::1686926799624::2478517360:295757165047::::::1686926361024::::::::::::::::::::2356359:162::::, false::1686926422993::2478517360:175753744878::::::1686926361024::::::::::::::::::::2356359:92::::, false::1686926779694::2478517360:175753744878::::::1686926361024::::::::::::::::::::2356359:135::::]",None,0
983924,2478517360,250482,0,101551,20230616,9f4be62eb40c4d46abe45c9e11101505,Apps: iPhone,1,134605253981,4,15709,,13000,13000,1,"[false::1686926799624::2478517360:295757165047::::::1686926361024::::::::::::::::::::2356359:162::::, false::1686926422993::2478517360:175753744878::::::1686926361024::::::::::::::::::::2356359:92::::, false::1686926779694::2478517360:175753744878::::::1686926361024::::::::::::::::::::2356359:135::::]",None,0
983925,2478517360,250482,0,101551,20230616,9f4be62eb40c4d46abe45c9e11101505,Apps: iPhone,1,204364007802,5,15709,,16000,13000,1,"[false::1686926799624::2478517360:295757165047::::::1686926361024::::::::::::::::::::2356359:162::::, false::1686926422993::2478517360:175753744878::::::1686926361024::::::::::::::::::::2356359:92::::, false::1686926779694::2478517360:175753744878::::::1686926361024::::::::::::::::::::2356359:135::::]",None,0
983926,2478517360,250482,0,101551,20230616,9f4be62eb40c4d46abe45c9e11101505,Apps: iPhone,1,295757165047,6,15709,,14000,13000,1,"[false::1686926799624::2478517360:295757165047::::::1686926361024::::::::::::::::::::2356359:162::::, false::1686926422993::2478517360:175753744878::::::1686926361024::::::::::::::::::::2356359:92::::, false::1686926779694::2478517360:175753744878::::::1686926361024::::::::::::::::::::2356359:135::::]",295757165047,1
983927,2478517360,250482,0,101551,20230616,9f4be62eb40c4d46abe45c9e11101505,Apps: iPhone,1,165484071928,7,15709,,14000,13000,1,"[false::1686926799624::2478517360:295757165047::::::1686926361024::::::::::::::::::::2356359:162::::, false::1686926422993::2478517360:175753744878::::::1686926361024::::::::::::::::::::2356359:92::::, false::1686926779694::2478517360:175753744878::::::1686926361024::::::::::::::::::::2356359:135::::]",None,0
983928,2478517360,250482,0,101551,20230616,9f4be62eb40c4d46abe45c9e11101505,Apps: iPhone,1,235019868063,8,15709,,17000,13000,1,"[false::1686926799624::2478517360:295757165047::::::1686926361024::::::::::::::::::::2356359:162::::, false::1686926422993::2478517360:175753744878::::::1686926361024::::::::::::::::::::2356359:92::::, false::1686926779694::2478517360:175753744878::::::1686926361024::::::::::::::::::::2356359:135::::]",None,0
983929,2478517360,250482,0,101551,20230616,9f4be62eb40c4d46abe45c9e11101505,Apps: iPhone,1,404212014870,9,15709,,11500,13000,1,"[false::1686926799624::2478517360:295757165047::::::1686926361024::::::::::::::::::::2356359:162::::, false::1686926422993::2478517360:175753744878::::::16869263610

### Load pretrainer data of evaluation 3/6-20/6 ab test dates 

In [26]:
!hdfs dfs -ls -h -C /apps/b_perso/hp/simplark/pretrainer/PersonalizedTopics*/2023/05*/

/apps/b_perso/hp/simplark/pretrainer/PersonalizedTopicsV2WithWatchlistFeaturesAndTopicMLR/2023/05/01
/apps/b_perso/hp/simplark/pretrainer/PersonalizedTopicsV2WithWatchlistFeaturesAndTopicMLR/2023/05/02
/apps/b_perso/hp/simplark/pretrainer/PersonalizedTopicsV2WithWatchlistFeaturesAndTopicMLR/2023/05/04
/apps/b_perso/hp/simplark/pretrainer/PersonalizedTopicsV2WithWatchlistFeaturesAndTopicMLR/2023/05/05
/apps/b_perso/hp/simplark/pretrainer/PersonalizedTopicsV2WithWatchlistFeaturesAndTopicMLR/2023/05/06
/apps/b_perso/hp/simplark/pretrainer/PersonalizedTopicsV2WithWatchlistFeaturesAndTopicMLR/2023/05/07
/apps/b_perso/hp/simplark/pretrainer/PersonalizedTopicsV2WithWatchlistFeaturesAndTopicMLR/2023/05/08
/apps/b_perso/hp/simplark/pretrainer/PersonalizedTopicsV2WithWatchlistFeaturesAndTopicMLR/2023/05/09
/apps/b_perso/hp/simplark/pretrainer/PersonalizedTopicsV2WithWatchlistFeaturesAndTopicMLR/2023/05/10
/apps/b_perso/hp/simplark/pretrainer/PersonalizedTopicsV2WithWatchlistFeaturesAndTopicMLR/2

In [27]:
!hdfs dfs -ls -h -C /apps/b_perso/hp/simplark/pretrainer/PersonalizedTopics*/2023/06*/

/apps/b_perso/hp/simplark/pretrainer/PersonalizedTopicsV2WithV7Unbiased/2023/06/03
/apps/b_perso/hp/simplark/pretrainer/PersonalizedTopicsV2WithV7Unbiased/2023/06/04
/apps/b_perso/hp/simplark/pretrainer/PersonalizedTopicsV2WithV7Unbiased/2023/06/05
/apps/b_perso/hp/simplark/pretrainer/PersonalizedTopicsV2WithV7Unbiased/2023/06/06
/apps/b_perso/hp/simplark/pretrainer/PersonalizedTopicsV2WithV7Unbiased/2023/06/07
/apps/b_perso/hp/simplark/pretrainer/PersonalizedTopicsV2WithV7Unbiased/2023/06/08
/apps/b_perso/hp/simplark/pretrainer/PersonalizedTopicsV2WithV7Unbiased/2023/06/09
/apps/b_perso/hp/simplark/pretrainer/PersonalizedTopicsV2WithV7Unbiased/2023/06/10
/apps/b_perso/hp/simplark/pretrainer/PersonalizedTopicsV2WithV7Unbiased/2023/06/11
/apps/b_perso/hp/simplark/pretrainer/PersonalizedTopicsV2WithV7Unbiased/2023/06/12
/apps/b_perso/hp/simplark/pretrainer/PersonalizedTopicsV2WithV7Unbiased/2023/06/13
/apps/b_perso/hp/simplark/pretrainer/PersonalizedTopicsV2WithV7Unbiased/2023/06/14
/app

In [18]:
!hdfs dfs -ls -h -C /apps/b_perso/hp/simplark/pretrainer/PersonalizedTopics*/2023/06/03/data/part-00000-1288443d-4fb0-4e97-b8c7-0dabba7a6c4a.npy.gz

/apps/b_perso/hp/simplark/pretrainer/PersonalizedTopicsV2WithV7Unbiased/2023/06/03/data/part-00000-1288443d-4fb0-4e97-b8c7-0dabba7a6c4a.npy.gz


### Load evaluation data:

In [76]:
sneakers_abtest_df=pd.read_pickle('/data/shpx/notebooks/olivyatan/Unbiasing/sneakers_size_abtest_transactions_290623_sample.pkl' )

In [77]:
sneakers_abtest_df.shape

(42979891, 18)

In [ ]:
https://github.ebay.com/olivyatan/evalplex/tree/krylov_week09_20230302124009_Thursday/htmls/piyi_v7 
https://github.ebay.com/olivyatan/evalplex/tree/krylov_week09_20230302124128_Thursday/htmls/piyi_v7_unbiased 
eval sneakers: 
    https://github.ebay.com/olivyatan/evalplex/tree/krylov_week27_20230706110041_Thursday/htmls/piyi_v7_sneakers_numpy

In [36]:
!ls -lh /data/shpx/data/olivyatan/week19_20230513163529_Saturday/simplex_models/simplex_piyi_v7_sneakers_numpy_week19_20230513134315_Saturday.pkl

-rw-r--r-- 1 root aip_shpx 126K May 13 13:44 /data/shpx/data/olivyatan/week19_20230513163529_Saturday/simplex_models/simplex_piyi_v7_sneakers_numpy_week19_20230513134315_Saturday.pkl


In [27]:
!ls -lh /data/shpx/data/olivyatan/week27_20230706110041_Thursday/prep/piyi_v7_sneakers_numpy_prepped_data

-rw-r--r-- 1 root aip_shpx 3.1M Jul  6 08:18 /data/shpx/data/olivyatan/week27_20230706110041_Thursday/prep/piyi_v7_sneakers_numpy_prepped_data


In [28]:
import pickle
import numpy as np
import pandas as pd
# link od model run: https://github.ebay.com/olivyatan/treeplex/tree/krylov_week07_20230213180248_Monday/htmls/piyi_v7_unbiased 
#https://github.ebay.com/olivyatan/treeplex/tree/krylov_week09_20230227132053_Monday/htmls/piyi_v7_unbiased 
##https://github.ebay.com/olivyatan/evalplex/tree/krylov_week10_20230308110625_Wednesday/htmls/piyi_v7_unbiased
file = open(f'/data/shpx/data/olivyatan/week27_20230706110041_Thursday/prep/piyi_v7_sneakers_numpy_prepped_data', 'rb')
# dump information to that file
prepped_data = pickle.load(file)
file.close()

In [29]:
np.unique(prepped_data.y_test, return_counts=True)

(array([0, 1], dtype=int32), array([8010,  953]))

In [87]:
len(prepped_data.schema)

36

In [86]:
prepped_data.schema

['BibowatchRelPosition',
 'RecallSourceBullseye',
 'RecallSourceTora',
 'TitleCosineSimilarityToShoppingcartCentroid',
 'MaxViewedItemTitleJaccardBigrams',
 'NumSameRviInLastWeek',
 'AvgSameLeafRviPriceRatio',
 'ItemSalesOverImpPricePrior7DayDecayLogSmoothDomesticWebAndMobile',
 'ItemVariantSalesOverImpressions7DayDecayLogSmoothDomesticWebAndMobileV2',
 'MaxViewedItemTitleJaccard',
 'ItemTimeOnSiteV2',
 'ItemWatchesOverImp7DayDecayLogSmoothDomesticWebAndMobileV2',
 'PriceDiffMedianRecall',
 'FreqSameItemInWatchBadge',
 'RecallSourceBestMatch',
 'ItemSalesOverImpPricePrior7DayDecayLogSmoothInternationalWebAndMobileNorm',
 'FreqWatchPriceBellowItemPrice',
 'MerchImpressionsDecayed',
 'PlImpressionsDecayed',
 'AvgSameLeafRviPriceDiff',
 'ItemSalesOverImpPricePrior7DayDecayLogSmoothDomesticWebAndMobileNorm',
 'BullseyeRelRVILeafCatMedianPriceDiffV2',
 'BullseyeAbsRVILeafCatMedianPriceDiffV2',
 'BullseyeRVILeafCatMedianPriceV2',
 'ItemTimeLeftSec',
 'ItemTimeOnSiteNorm',
 'ItemWatchOverImpL

In [53]:

f=pd.DataFrame(prepped_data.X_test)

In [54]:
f

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,-1.000,1.000,1.000,-1.000,0.250,0.000,0.749,-7.591,-7.335,0.250,...,0.541,0.000,0.647,1.461,0.127,0.005,0.000,0.000,6.000,24.000
1,-1.000,1.000,1.000,-1.000,0.143,1.000,0.622,-12.386,-11.995,0.227,...,0.585,0.001,0.688,0.892,0.472,0.004,0.000,0.000,6.000,24.000
2,-1.000,1.000,1.000,-1.000,0.286,1.000,1.390,-10.129,-10.433,0.389,...,0.651,0.000,0.553,1.496,0.316,0.017,0.000,0.000,6.000,24.000
3,-1.000,1.000,1.000,-1.000,0.050,0.000,0.911,-9.068,-11.151,0.059,...,0.384,0.000,0.596,0.981,0.159,0.001,0.000,0.000,6.000,24.000
4,-1.000,0.000,1.000,-1.000,0.182,0.000,1.031,-7.677,-7.335,0.333,...,0.541,0.000,0.647,1.220,0.377,0.003,0.000,0.000,6.000,24.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8958,-1.000,0.000,1.000,0.000,0.077,0.000,1.355,-10.301,-12.500,0.000,...,0.688,0.001,0.635,1.226,0.349,0.001,0.000,1.000,1.500,34.000
8959,-1.000,0.000,1.000,0.000,0.075,0.000,1.074,-11.730,-11.983,0.083,...,0.627,0.001,0.632,1.022,0.335,0.029,0.000,1.500,1.500,34.000
8960,-1.000,0.000,1.000,0.000,0.071,0.000,0.510,-10.848,-13.145,0.077,...,0.582,0.002,0.568,0.403,0.433,0.000,0.000,1.500,1.500,34.000
8961,-1.000,0.000,1.000,0.000,0.000,0.000,1.074,-10.272,-10.983,0.000,...,0.549,0.000,0.469,1.520,0.472,0.000,0.000,2.500,1.500,34.000


In [39]:
pdf = pd.DataFrame(prepped_data.hold_out)
pdf.head(10)

,meid,mlrRank,itemId,labels.labelClassicBBE
0,b'2c1964b7ba30479fb52c48d47dd3260b',1,314651024057,-1
1,b'2c1964b7ba30479fb52c48d47dd3260b',2,185646232849,-1
2,b'2c1964b7ba30479fb52c48d47dd3260b',3,195650041295,-1
3,b'2c1964b7ba30479fb52c48d47dd3260b',4,385678149660,-1
4,b'2c1964b7ba30479fb52c48d47dd3260b',5,175778880383,-1
5,b'2c1964b7ba30479fb52c48d47dd3260b',6,195830685887,-1
6,b'2c1964b7ba30479fb52c48d47dd3260b',7,185949473399,-1
7,b'2c1964b7ba30479fb52c48d47dd3260b',8,185938182902,-1
8,b'2c1964b7ba30479fb52c48d47dd3260b',9,314650534934,-1
9,b'2c1964b7ba30479fb52c48d47dd3260b',10,225589218458,-1


In [40]:
pdf.shape

(8963, 4)

In [43]:
dir(prepped_data)

['X_test',
 'X_test_for_calibration',
 'X_test_sample_weight',
 'X_test_sample_weight_eval',
 'X_train',
 'X_train_sample_weight',
 'X_train_sample_weight_eval',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'apply_column_filters',
 'apply_data_filters',
 'apply_filters',
 'compute_event_rate',
 'compute_multi_label_event_rate',
 'construct_label_array',
 'event_rate',
 'extract_schema',
 'extract_weights',
 'filter_examples',
 'force_contiguous',
 'get_eval_weights',
 'get_event_rates',
 'get_label_column',
 'group_hold_out',
 'group_train',
 'hold_out',
 'mem_usage',
 'raw_event_rate',
 'sample_data',
 'schema',
 'set_click_propensities',
 'set_up_logger',
 'split_

In [46]:
prepped_data.X_test_sample_weight

array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)

In [47]:
len(prepped_data.y_test)

8963

In [48]:
prepped_data.X_test_sample_weight

array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)

In [49]:
prepped_data.X_test.shape

(8963, 36)

In [50]:
prepped_data.hold_out

{'meid': array([b'2c1964b7ba30479fb52c48d47dd3260b',
        b'2c1964b7ba30479fb52c48d47dd3260b',
        b'2c1964b7ba30479fb52c48d47dd3260b', ...,
        b'9bfd09ccd0d346029ec71b9ce20e1559',
        b'9bfd09ccd0d346029ec71b9ce20e1559',
        b'9bfd09ccd0d346029ec71b9ce20e1559'], dtype='|S32'),
 'mlrRank': array([ 1,  2,  3, ...,  8,  9, 10], dtype=int32),
 'itemId': array([314651024057, 185646232849, 195650041295, ..., 115815096578,
        275758542303, 114936287317]),
 'labels.labelClassicBBE': array([-1, -1, -1, ..., -1, -1, -1], dtype=int32)}

In [51]:
import xgboost
xgb_model_loaded = xgboost.Booster()
xgb_model_loaded.load_model('sneakers_14.5.23_model.bst')

[11:35:55] WARNING: ../src/learner.cc:851: Loading model from XGBoost < 1.0.0, consider saving it again for improved compatibility


In [58]:
y_pred = xgb_model_loaded.predict(xgboost.DMatrix(prepped_data.X_test))

pdf['y_pred'] = y_pred 
pdf['rank_sneakers'] = pdf.groupby('meid')['y_pred'].rank(method='dense', ascending=False)

In [59]:
pdf['label'] = prepped_data.y_test

In [60]:
pdf.sort_values(by=['meid','mlrRank'],inplace=True)
pdf.drop([ 'labels.labelClassicBBE', ], axis=1,inplace=True)

In [68]:
pdf.head(10)

,meid,mlrRank,itemId,y_pred,rank_sneakers,label,meid_str
8467,b'001dec54031647b49ab8cc7708757375',1,404313698485,1.892,1.000,1,001dec54031647b49ab8cc7708757375
8468,b'001dec54031647b49ab8cc7708757375',2,404312296105,1.188,2.000,0,001dec54031647b49ab8cc7708757375
8469,b'001dec54031647b49ab8cc7708757375',3,404290477347,-0.574,7.000,0,001dec54031647b49ab8cc7708757375
8470,b'001dec54031647b49ab8cc7708757375',4,275878859761,1.107,3.000,0,001dec54031647b49ab8cc7708757375
8471,b'001dec54031647b49ab8cc7708757375',6,275605855825,0.665,5.000,0,001dec54031647b49ab8cc7708757375
8472,b'001dec54031647b49ab8cc7708757375',7,403785890294,0.700,4.000,0,001dec54031647b49ab8cc7708757375
8473,b'001dec54031647b49ab8cc7708757375',8,185906785999,-1.299,8.000,0,001dec54031647b49ab8cc7708757375
8474,b'001dec54031647b49ab8cc7708757375',9,285133428180,0.554,6.000,0,001dec54031647b49ab8cc7708757375
7146,b'00397eabeead40dbbab0e5360300ea38',1,314620856822,0.568,10.000,1,00397eabeead40dbbab0e5360300ea38
7147,b'00397eabeead40dbbab0e5360300ea38',2,354754711702,1.344,3.000,0,00397eabeead40dbbab0e5360300ea38


In [72]:
pdf.meid_str

8467    001dec54031647b49ab8cc7708757375
8468    001dec54031647b49ab8cc7708757375
8469    001dec54031647b49ab8cc7708757375
8470    001dec54031647b49ab8cc7708757375
8471    001dec54031647b49ab8cc7708757375
                      ...               
5379    fff552e4a1e74780b087702e1be94ade
5380    fff552e4a1e74780b087702e1be94ade
5381    fff552e4a1e74780b087702e1be94ade
5382    fff552e4a1e74780b087702e1be94ade
5383    fff552e4a1e74780b087702e1be94ade
Name: meid_str, Length: 8963, dtype: object

In [70]:
pdf[pdf['meid_str']=='001dec54031647b49ab8cc7708757375']

,meid,mlrRank,itemId,y_pred,rank_sneakers,label,meid_str
8467,b'001dec54031647b49ab8cc7708757375',1,404313698485,1.892,1.000,1,001dec54031647b49ab8cc7708757375
8468,b'001dec54031647b49ab8cc7708757375',2,404312296105,1.188,2.000,0,001dec54031647b49ab8cc7708757375
8469,b'001dec54031647b49ab8cc7708757375',3,404290477347,-0.574,7.000,0,001dec54031647b49ab8cc7708757375
8470,b'001dec54031647b49ab8cc7708757375',4,275878859761,1.107,3.000,0,001dec54031647b49ab8cc7708757375
8471,b'001dec54031647b49ab8cc7708757375',6,275605855825,0.665,5.000,0,001dec54031647b49ab8cc7708757375
8472,b'001dec54031647b49ab8cc7708757375',7,403785890294,0.700,4.000,0,001dec54031647b49ab8cc7708757375
8473,b'001dec54031647b49ab8cc7708757375',8,185906785999,-1.299,8.000,0,001dec54031647b49ab8cc7708757375
8474,b'001dec54031647b49ab8cc7708757375',9,285133428180,0.554,6.000,0,001dec54031647b49ab8cc7708757375


In [71]:
pdf[pdf['meid_str']=='00397eabeead40dbbab0e5360300ea38']

,meid,mlrRank,itemId,y_pred,rank_sneakers,label,meid_str
7146,b'00397eabeead40dbbab0e5360300ea38',1,314620856822,0.568,10.000,1,00397eabeead40dbbab0e5360300ea38
7147,b'00397eabeead40dbbab0e5360300ea38',2,354754711702,1.344,3.000,0,00397eabeead40dbbab0e5360300ea38
7148,b'00397eabeead40dbbab0e5360300ea38',3,314625118332,1.635,1.000,0,00397eabeead40dbbab0e5360300ea38
7149,b'00397eabeead40dbbab0e5360300ea38',4,385635623035,1.376,2.000,0,00397eabeead40dbbab0e5360300ea38
7150,b'00397eabeead40dbbab0e5360300ea38',5,225543634679,1.082,4.000,0,00397eabeead40dbbab0e5360300ea38
7151,b'00397eabeead40dbbab0e5360300ea38',6,175753658467,0.824,8.000,0,00397eabeead40dbbab0e5360300ea38
7152,b'00397eabeead40dbbab0e5360300ea38',7,225584168489,1.026,7.000,0,00397eabeead40dbbab0e5360300ea38
7153,b'00397eabeead40dbbab0e5360300ea38',8,204335504214,1.033,5.000,0,00397eabeead40dbbab0e5360300ea38
7154,b'00397eabeead40dbbab0e5360300ea38',9,404290992548,1.032,6.000,0,00397eabeead40dbbab0e5360300ea38
7155,b'00397eabeead40dbbab0e5360300ea38',10,325381930396,0.798,9.000,0,00397eabeead40dbbab0e5360300ea38


In [73]:
pdf[pdf['meid_str']=='fff552e4a1e74780b087702e1be94ade']

,meid,mlrRank,itemId,y_pred,rank_sneakers,label,meid_str
5374,b'fff552e4a1e74780b087702e1be94ade',1,195806177833,0.898,3.000,0,fff552e4a1e74780b087702e1be94ade
5375,b'fff552e4a1e74780b087702e1be94ade',2,394670791766,2.060,1.000,0,fff552e4a1e74780b087702e1be94ade
5376,b'fff552e4a1e74780b087702e1be94ade',3,204358207530,0.814,4.000,1,fff552e4a1e74780b087702e1be94ade
5377,b'fff552e4a1e74780b087702e1be94ade',4,204288585733,-0.240,6.000,0,fff552e4a1e74780b087702e1be94ade
5378,b'fff552e4a1e74780b087702e1be94ade',5,285319517527,-1.329,10.000,0,fff552e4a1e74780b087702e1be94ade
5379,b'fff552e4a1e74780b087702e1be94ade',6,204362613581,0.914,2.000,0,fff552e4a1e74780b087702e1be94ade
5380,b'fff552e4a1e74780b087702e1be94ade',7,204356043265,-0.642,9.000,0,fff552e4a1e74780b087702e1be94ade
5381,b'fff552e4a1e74780b087702e1be94ade',8,155585121469,0.659,5.000,0,fff552e4a1e74780b087702e1be94ade
5382,b'fff552e4a1e74780b087702e1be94ade',9,204359935944,-0.534,8.000,0,fff552e4a1e74780b087702e1be94ade
5383,b'fff552e4a1e74780b087702e1be94ade',10,235044591972,-0.323,7.000,0,fff552e4a1e74780b087702e1be94ade


In [62]:
pdf['meid_str']=pdf['meid'].apply(lambda x: x.decode("utf-8") )

In [63]:
pdf[pdf['label']==1]['mlrRank'].mean()

2.640083945435467

In [64]:
pdf[pdf['label']==1]['rank_sneakers'].mean()

2.1668415529905563

In [74]:
pdf[['meid','meid_str', 'itemId']].dtypes

meid        object
meid_str    object
itemId       int64
dtype: object

In [78]:
print(sneakers_abtest_df[['meid', 'itemId']].dtypes)

meid      object
itemId    object
dtype: object


In [79]:
pdf["itemId"] = pdf["itemId"].astype("int64") 
sneakers_abtest_df["itemId"] = sneakers_abtest_df["itemId"].astype("int64") 

In [80]:
pdf['meid_str'] = pdf['meid_str'].astype(str)
sneakers_abtest_df['meid'] = sneakers_abtest_df['meid'].astype(str)

In [81]:

pdf_abtest_eval = pd.merge(sneakers_abtest_df, pdf, how ='inner', left_on=['itemId','meid']  , right_on =['itemId','meid_str'])


In [82]:
pdf_abtest_eval.shape

(969, 24)

In [83]:
pdf_abtest_eval.head(2)

,user_id,TRKS,SITE_ID,plmt,dt,meid_x,Experience,SURFACE_PLMT_IMP,itemId,production_rank,...,plmt_purchase_1d,plmt_clicks,purchased_itemId,is_purchase,meid_y,mlrRank,y_pred,rank_sneakers,label,meid_str
0,432425295,250483,0,101251,20230613,a68350b78b29463fb8f8063205efb8b0,Apps: iPhone,1,185930313654,1,...,0,[false::1686666426023::432425295:185930313654::::::1686666408246::::::::::::::::::::2356359:48::::],None,0,b'a68350b78b29463fb8f8063205efb8b0',1,2.878,1.000,1,a68350b78b29463fb8f8063205efb8b0
1,432425295,250483,0,101251,20230613,a68350b78b29463fb8f8063205efb8b0,Apps: iPhone,1,304973610508,2,...,0,[false::1686666426023::432425295:185930313654::::::1686666408246::::::::::::::::::::2356359:48::::],None,0,b'a68350b78b29463fb8f8063205efb8b0',2,-0.860,7.000,0,a68350b78b29463fb8f8063205efb8b0


In [84]:
pdf_abtest_eval.columns

Index(['user_id', 'TRKS', 'SITE_ID', 'plmt', 'dt', 'meid_x', 'Experience',
       'SURFACE_PLMT_IMP', 'itemId', 'production_rank', 'leafCategoryId',
       'clicks', 'price', 'plmt_gmv_sum_1d', 'plmt_purchase_1d', 'plmt_clicks',
       'purchased_itemId', 'is_purchase', 'meid_y', 'mlrRank', 'y_pred',
       'rank_sneakers', 'label', 'meid_str'],
      dtype='object')

In [ ]:
pdf_abtest_eval[pdf_abtest_eval['label']==1]['rank_sneakers'].mean()

In [ ]:
pdf_abtest_eval['rank_change_mlrank']=pdf_abtest_eval['mlrRank']-pdf_abtest_eval['rank_sneakers']
pdf_abtest_eval['rank_change_madcdl']=pdf_abtest_eval['production_rank']-pdf_abtest_eval['rank_sneakers']

In [ ]:
pdf_abtest_eval['rank_change_mlrank'].mean()

In [ ]:
pdf_abtest_eval['ranks_are_equal_mlrank'] = pdf_abtest_eval.apply(lambda x: int(x['mlrRank'] ==x['rank_sneakers']), axis=1)

In [ ]:
pdf_abtest_eval[pdf_abtest_eval['is_purchase']>0]['mlrRank'].mean()